In [4]:
import pandas as pd
import numpy as np
import os
os.chdir(r'D:\Old Account\Ecomm\Oechsle\Raw data\FB BB')

df = pd.read_excel('Data_Oechsle_MMM_VF.xlsx',sheet_name='Sheet4')


#Label creation
df1 = df.pivot_table(index='Identificador de la campaña',values=['Duration','Frecuencia','Reach'],
                     aggfunc={'Duration':'mean','Frecuencia':'mean','Reach':'sum'}).reset_index()
df1['Avg Freq'] = df1['Frecuencia']/df1['Duration']
df1['Reach %'] = df1['Reach']/33000000

df1['Duration Label'] = np.where(df1['Duration']<2,'Sub optimal',np.where(df1['Duration'].between(2,4),'Opportunity to improve',
                                                                         'Optimal'))
df1['Frequency Label'] = np.where(df1['Avg Freq']<1,'Sub optimal','Optimal')
df1['Reach Label'] = np.where(df1['Reach %']<=0.2,'Sub optimal',np.where(df1['Reach %'].between(0.2,0.4),'Opportunity to improve',
                                                                         'Optimal'))
df1.drop(df1.columns[1:6],1,inplace=True)

df2 = pd.merge(df,df1,on='Identificador de la campaña', how='left')

C:\Users\GowthamPSabareesan\AppData\Local\Temp\ipykernel_10320\3922612611.py:20: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df1.drop(df1.columns[1:6],1,inplace=True)


In [5]:
df3 = pd.read_csv('Oeschle_FB_data_country_23.04.2022.csv',parse_dates=['Date'])
test = df3.drop_duplicates(['Campaign ID','Placement'])
so = ['Link clicks','Post engagement']
op = ['Reach','Brand awareness','Video views','Conversions','Product catalog sales','Traffic','Store traffic','Store visits']
df3['Objective Label']= np.where(df3['Campaign Objective'].isin(op),'Optimal',
                                 np.where(df3['Campaign Objective'].isin(so),'Sub Optimal','Opportunity to improve'))
df3['Placement Count'] = df3['Campaign ID'].map(test['Campaign ID'].value_counts().to_dict())
df3['Placement Label'] = np.where(df3['Placement Count']<4,'Sub Optimal','Optimal')
df3.rename(columns={'Campaign ID':'Identificador de la campaña'},inplace=True)
df03 = df3.drop_duplicates('Identificador de la campaña')

In [6]:
df4 = pd.merge(df2,df03[['Identificador de la campaña','Objective Label','Placement Label']],on='Identificador de la campaña',
              how='left')
for i in df4.columns[-5:]:
    df4[i + ' points'] = np.where(df4[i]=='Optimal',2, np.where(df4[i]=='Sub Optimal',0,1))
df4['Total Points'] = df4[df4.columns[-5:]].sum(1)
df4['Total Label'] = np.where(df4['Total Points']<=3,'Sub Optimal',
                              np.where(df4['Total Points'].between(4,7),'Opprotunity to improve','Optimal'))
df4.to_excel('Oechsle_Processed_Meta_BB_data_16Oct.xlsx')